<a href="https://colab.research.google.com/github/MeghanaAkkineni/MeghanaAkkineni/blob/main/podcast_summerization_and_translator_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip uninstall -y whisper -q
!pip install git+https://github.com/openai/whisper.git -q
!pip install transformers pydub gTTS ipywidgets -q
!apt-get install -y ffmpeg -qq
import whisper
from transformers import pipeline
from pydub import AudioSegment
from gtts import gTTS
from IPython.display import display, HTML, Audio
import ipywidgets as widgets
import tempfile
display(HTML("<h3 style='color:#7D3C98;'>🔄 Loading models, please wait...</h3>"))
whisper_model = whisper.load_model("tiny")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
file_upload = widgets.FileUpload(accept='.mp3,.wav', multiple=False, description="📁 Upload Audio")
language_dropdown = widgets.Dropdown(
    options=['English', 'Hindi', 'French', 'Spanish', 'German'],
    value='Hindi',
    description='🌍 Translate to:',
    style={'description_width': 'initial'}
)
transcribe_button = widgets.Button(
    description="🎧 Generate Summary & Translate",
    button_style='info',
    layout=widgets.Layout(width='auto'),
    style={'button_color': '#3498db'}
)
output_area = widgets.Output()
display(HTML("""
<div style='background:#2E86C1;padding:20px;border-radius:10px;color:white;text-align:center'>
    <h2>🎙️ Podcast Summarizer & Translator</h2>
    <p>Upload an audio file, get a transcript, summary, and hear it in your selected language!</p>
</div><br>
"""))
display(file_upload, language_dropdown, transcribe_button, output_area)
def translate_text(text, target_lang):
    lang_map = {
        "English": "en", "Hindi": "hi", "French": "fr",
        "Spanish": "es", "German": "de"
    }
    lang_code = lang_map.get(target_lang, "en")
    if lang_code == "en":
        return text, lang_code
    else:
        trans_pipe = pipeline("translation_en_to_" + lang_code, model=f"Helsinki-NLP/opus-mt-en-{lang_code}")
        translated = trans_pipe(text)[0]['translation_text']
        return translated, lang_code
def on_transcribe_clicked(b):
    output_area.clear_output()
    if not file_upload.value:
        with output_area:
            display(HTML("<p style='color:red;'>❗ Please upload an audio file.</p>"))
        return
    file_name = list(file_upload.value.keys())[0]
    content = file_upload.value[file_name]['content']
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_file:
        temp_file.write(content)
        audio_path = temp_file.name
    audio = AudioSegment.from_file(audio_path)
    audio = audio[:60 * 1000]
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as wav_file:
        audio.export(wav_file.name, format="wav")
        wav_path = wav_file.name
    with output_area:
        display(HTML("<h4 style='color:#2980b9;'>🔍 Transcribing audio...</h4>"))
        result = whisper_model.transcribe(wav_path)
        transcript = result["text"]
        display(HTML("<h4 style='color:#117864;'>📄 Transcript:</h4>"))
        display(HTML(f"<div style='border:1px solid #ccc;padding:10px;background:#EAFAF1;border-radius:6px'>{transcript}</div>"))
        display(HTML("<h4 style='color:#D35400;'>📝 Summarizing...</h4>"))
        chunks = [transcript[i:i+1000] for i in range(0, len(transcript), 1000)]
        summary_text = ""
        for chunk in chunks:
            summary_chunk = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
            summary_text += summary_chunk + " "
        display(HTML("<h4 style='color:#BA4A00;'>✨ Summary:</h4>"))
        display(HTML(f"<div style='border:1px solid #ccc;padding:10px;background:#FEF5E7;border-radius:6px'>{summary_text.strip()}</div>"))
        target_lang = language_dropdown.value
        display(HTML(f"<h4 style='color:#6C3483;'>🌐 Translating to {target_lang}...</h4>"))
        translated_text, lang_code = translate_text(summary_text, target_lang)
        display(HTML(f"<h4 style='color:#6C3483;'>📘 Translated Summary ({target_lang}):</h4>"))
        display(HTML(f"<div style='border:1px solid #ccc;padding:10px;background:#F5EEF8;border-radius:6px'>{translated_text}</div>"))
        display(HTML("<h4 style='color:#3498DB;'>🔊 Generating Translated Audio...</h4>"))
        tts = gTTS(translated_text, lang=lang_code)
        audio_path_out = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3").name
        tts.save(audio_path_out)
        display(Audio(audio_path_out))
        display(HTML(f"<a href='{audio_path_out}' download='translated_summary.mp3'>⬇️ <b>Download Translated Audio</b></a>"))

transcribe_button.on_click(on_transcribe_clicked)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Device set to use cpu


FileUpload(value={}, accept='.mp3,.wav', description='📁 Upload Audio')

Dropdown(description='🌍 Translate to:', index=1, options=('English', 'Hindi', 'French', 'Spanish', 'German'), …

Button(button_style='info', description='🎧 Generate Summary & Translate', layout=Layout(width='auto'), style=B…

Output()